In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

from model import ResNet

import numpy as np
from main import ChunkSampler, get_param_count, check_accuracy


In [5]:
transform_augment = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(32, padding=4)])
transform_normalize = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
# get CIFAR-10 data
NUM_TRAIN = 45000
NUM_VAL = 5000
batch_size = 128
cifar10_train = dset.CIFAR10('./dataset', train=True, download=True,
                                transform=T.Compose([transform_augment, transform_normalize]))
loader_train = DataLoader(cifar10_train, batch_size=batch_size,
                            sampler=ChunkSampler(NUM_TRAIN))
cifar10_val = dset.CIFAR10('./dataset', train=True, download=True,
                            transform=transform_normalize)
loader_val = DataLoader(cifar10_train, batch_size=batch_size,
                        sampler=ChunkSampler(NUM_VAL, start=NUM_TRAIN))
cifar10_test = dset.CIFAR10('./dataset', train=False, download=True,
                            transform=transform_normalize)
loader_test = DataLoader(cifar10_test, batch_size=batch_size)


100.0%


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified
Files already downloaded and verified


In [6]:
n = 5
res_option = 'A'
weight_decay = 0.0001
use_dropout = False
learning_rate = 0.1

In [8]:
# load model
model = ResNet(n, res_option=res_option, use_dropout=use_dropout)

param_count = get_param_count(model)
print('Parameter count: %d' % param_count)

# use gpu for training
if not torch.cuda.is_available():
    print('Error: CUDA library unavailable on system')
    
global gpu_dtype
gpu_dtype = torch.cuda.FloatTensor
model = model.type(gpu_dtype)

# setup loss function
criterion = nn.CrossEntropyLoss().cuda()
# train model
SCHEDULE_EPOCHS = [50, 5, 5] # divide lr by 10 after each number of epochs
#     SCHEDULE_EPOCHS = [100, 50, 50] # divide lr by 10 after each number of epochs


Parameter count: 465290


In [ ]:

for num_epochs in SCHEDULE_EPOCHS:
    print('Training for %d epochs with learning rate %f' % (num_epochs, learning_rate))
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                            momentum=0.9, weight_decay=weight_decay)
    for epoch in range(num_epochs):
        check_accuracy(model, loader_val)
        print('Starting epoch %d / %d' % (epoch+1, num_epochs))
        train(loader_train, model, criterion, optimizer)
    learning_rate *= 0.1

print('Final test accuracy:')
check_accuracy(model, loader_test)